# Working with the CIA World Factbook

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [3]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [5]:
%%sql
SELECT * FROM facts LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


To Begin, I want to calculate some summary statistics and look for any outliers. I'll look at the Min pop, Max pop, Min pop growth, and Max pop growth.

In [6]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth) FROM facts

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


I can see that a few things stick out; there's a country with a minimum population of zero, and a country with a maximum population over 7.2 billion. I'll take a closer look to see if I can figure out what's going on.

In [13]:
%%sql
SELECT name, population FROM facts WHERE population = 0

Done.


name,population
Antarctica,0


In [14]:
%%sql
SELECT name, population FROM facts WHERE population = 7256490011

Done.


name,population
World,7256490011


It looks like Antartica has a population of 0. There is also a row in this dataset for the whole world, which supplies the surprising max population data.


# Small and densely populated countries
Let's look at some averages.

In [16]:
%%sql
SELECT AVG(population), AVG(area) FROM facts

Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


Let's explore what countries are small and densely populated. The query below finds countries where the population is above average, and the area is below average.

In [19]:
%%sql
SELECT name, population, area FROM facts WHERE (population > (SELECT AVG(population) FROM facts)) AND (area < (SELECT AVG(area) FROM facts))




Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210


Above we can see the chart of the countries that matched!

# Highest ratio of water to land

Let's take a look at which countries have the highest ratios of water to land. The query below gives the top five countries with the highest water to land ratio. It looks like many of the countries have no water areae listed however, so we only have two numbers.

In [28]:
%%sql
SELECT name, (cast(area_water AS float)) / (cast(area_land AS float)) ratio FROM facts ORDER BY ratio DESC LIMIT 5 

Done.


name,ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623


# Countries with more water than land

Are there any countries with more water than land? This should be the ones with ratios > 1, but lets double check with a query.

In [24]:
%%sql
SELECT name, area_water, area_land FROM facts where area_water > area_land

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


Looks like these two meet the criteria!

# Which countries will add the most people to their population next year?

The population_growth column represents the growth rate as a percentage. Let's do some multiplication!

In [33]:
%%sql
SELECT name, ((population_growth/100) * population) added_pop FROM facts ORDER BY added_pop DESC LIMIT 7 

Done.


name,added_pop
World,78370092.1188
India,15270686.124799998
China,6153684.246
Nigeria,4448270.372
Pakistan,2906653.3662
Ethiopia,2874562.1691
Bangladesh,2703323.92


Ignoring the world row, we can see which countries are forcasted to add the most people to their population next year.

# Higher death rate than birth rate

Which countries have a higher death rate than birth rate?

In [35]:
%%sql
SELECT name, death_rate, birth_rate, population_growth FROM facts WHERE death_rate > birth_rate 

Done.


name,death_rate,birth_rate,population_growth
Austria,9.42,9.41,0.55
Belarus,13.36,10.7,0.2
Bosnia and Herzegovina,9.75,8.87,0.13
Bulgaria,14.44,8.92,0.58
Croatia,12.18,9.45,0.13
Czech Republic,10.34,9.63,0.16
Estonia,12.4,10.51,0.55
Germany,11.42,8.47,0.17
Greece,11.09,8.66,0.01
Hungary,12.73,9.16,0.22


Here's a list of the countries where the death rate is greater than the birth rate.

I wanted to look at the population growth for these countries as well, as I was curious if the population growth was calculated based on solely the death and birth rates. We can see that the differences between the rates don't correspond proportionately with the population growth rates however, so the growth rates must be calculated differently. 